# 02-1 API 사용하기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/hg-da/blob/main/02-1.ipynb"><img src="https://jupyter.org/assets/share.png" width="61" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-da/blob/main/02-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## 파이썬에서 JSON 데이터 다루기

In [1]:
d = {"name": "혼자 공부하는 데이터 분석"}

print(d['name'])

혼자 공부하는 데이터 분석


In [2]:
import json

In [3]:
d_str = json.dumps(d, ensure_ascii=False)
print(d_str)

{"name": "혼자 공부하는 데이터 분석"}


In [4]:
print(type(d_str))

<class 'str'>


In [5]:
d2 = json.loads(d_str)

print(d2['name'])

혼자 공부하는 데이터 분석


In [6]:
print(type(d2))

<class 'dict'>


In [7]:
d3 = json.loads('{"name": "혼자 공부하는 데이터 분석", "author": "박해선", "year": 2022}')

print(d3['name'])
print(d3['author'])
print(d3['year'])

혼자 공부하는 데이터 분석
박해선
2022


In [8]:
d3 = json.loads('{"name": "혼자 공부하는 데이터 분석", "author": ["박해선","홍길동"], \
                  "year": 2022}')

print(d3['author'][1])

홍길동


In [9]:
d4_str = """
[
  {"name": "혼자 공부하는 데이터 분석", "author": "박해선", "year": 2022},
  {"name": "혼자 공부하는 머신러닝+딥러닝", "author": "박해선", "year": 2020}
]
"""
d4 = json.loads(d4_str)

print(d4[0]['name'])

혼자 공부하는 데이터 분석


In [10]:
import pandas as pd

pd.read_json(d4_str)

,name,author,year
0,혼자 공부하는 데이터 분석,박해선,2022
1,혼자 공부하는 머신러닝+딥러닝,박해선,2020


In [11]:
pd.DataFrame(d4)

,name,author,year
0,혼자 공부하는 데이터 분석,박해선,2022
1,혼자 공부하는 머신러닝+딥러닝,박해선,2020


## 파이썬에서 XML 다루기

In [12]:
x_str = """
<book>
    <name>혼자 공부하는 데이터 분석</name>
    <author>박해선</author>
    <year>2022</year>
</book>
"""

In [13]:
import xml.etree.ElementTree as et

book = et.fromstring(x_str)

In [14]:
print(type(book))

<class 'xml.etree.ElementTree.Element'>


In [15]:
print(book.tag)

book


In [16]:
book_childs = list(book)

print(book_childs)

[<Element 'name' at 0x79328e929580>, <Element 'author' at 0x79328e9295d0>, <Element 'year' at 0x79328e929620>]


In [17]:
name, author, year = book_childs

print(name.text)
print(author.text)
print(year.text)

혼자 공부하는 데이터 분석
박해선
2022


In [18]:
name = book.findtext('name')
author = book.findtext('author')
year = book.findtext('year')

print(name)
print(author)
print(year)

혼자 공부하는 데이터 분석
박해선
2022


In [19]:
x2_str = """
<books>
    <book>
        <name>혼자 공부하는 데이터 분석</name>
        <author>박해선</author>
        <year>2022</year>
    </book>
    <book>
        <name>혼자 공부하는 머신러닝+딥러닝</name>
        <author>박해선</author>
        <year>2020</year>
    </book>
</books>
"""

In [20]:
books = et.fromstring(x2_str)

print(books.tag)

books


In [21]:
for book in books.findall('book'):
    name = book.findtext('name')
    author = book.findtext('author')
    year = book.findtext('year')

    print(name)
    print(author)
    print(year)
    print()

혼자 공부하는 데이터 분석
박해선
2022

혼자 공부하는 머신러닝+딥러닝
박해선
2020



In [22]:
pd.read_xml(x2_str)

,name,author,year
0,혼자 공부하는 데이터 분석,박해선,2022
1,혼자 공부하는 머신러닝+딥러닝,박해선,2020


## 파이썬으로 API 호출하기

In [23]:
import requests

In [24]:
# 인증키를 발급받아 문자열 맨 끝에 추가해 주세요.
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2021-04-01&endDt=2021-04-30&age=20&authKey=c01ec15e4574f74ee45cba2601bad15b82971e606e3b0740977ee4b363ce2fe2"

In [25]:
r = requests.get(url)

In [26]:
data = r.json()

print(data)

{'response': {'request': {'startDt': '2021-04-01', 'endDt': '2021-04-30', 'age': '20', 'pageNo': 1, 'pageSize': 200}, 'resultNum': 200, 'numFound': 5000, 'docs': [{'doc': {'no': 1, 'ranking': '1', 'bookname': '우리가 빛의 속도로 갈 수 없다면 :김초엽 소설 ', 'authors': '지은이: 김초엽', 'publisher': '허블', 'publication_year': '2019', 'isbn13': '9791190090018', 'addition_symbol': '03810', 'vol': '', 'class_no': '813.7', 'class_nm': '문학 > 한국문학 > 소설', 'loan_count': '487', 'bookImageURL': 'https://image.aladin.co.kr/product/19359/16/cover/s972635417_1.jpg', 'bookDtlUrl': 'https://data4library.kr/bookV?seq=5430429'}}, {'doc': {'no': 2, 'ranking': '2', 'bookname': '달러구트 꿈 백화점 :이미예 장편소설', 'authors': '지은이: 이미예', 'publisher': '팩토리나인', 'publication_year': '2020', 'isbn13': '9791165341909', 'addition_symbol': '03810', 'vol': '', 'class_no': '813.7', 'class_nm': '문학 > 한국문학 > 소설', 'loan_count': '407', 'bookImageURL': 'https://image.aladin.co.kr/product/24512/70/cover/k392630952_1.jpg', 'bookDtlUrl': 'https://data4library.kr

In [27]:
data

{'response': {'request': {'startDt': '2021-04-01',
   'endDt': '2021-04-30',
   'age': '20',
   'pageNo': 1,
   'pageSize': 200},
  'resultNum': 200,
  'numFound': 5000,
  'docs': [{'doc': {'no': 1,
     'ranking': '1',
     'bookname': '우리가 빛의 속도로 갈 수 없다면 :김초엽 소설 ',
     'authors': '지은이: 김초엽',
     'publisher': '허블',
     'publication_year': '2019',
     'isbn13': '9791190090018',
     'addition_symbol': '03810',
     'vol': '',
     'class_no': '813.7',
     'class_nm': '문학 > 한국문학 > 소설',
     'loan_count': '487',
     'bookImageURL': 'https://image.aladin.co.kr/product/19359/16/cover/s972635417_1.jpg',
     'bookDtlUrl': 'https://data4library.kr/bookV?seq=5430429'}},
   {'doc': {'no': 2,
     'ranking': '2',
     'bookname': '달러구트 꿈 백화점 :이미예 장편소설',
     'authors': '지은이: 이미예',
     'publisher': '팩토리나인',
     'publication_year': '2020',
     'isbn13': '9791165341909',
     'addition_symbol': '03810',
     'vol': '',
     'class_no': '813.7',
     'class_nm': '문학 > 한국문학 > 소설',
     'loa

In [28]:
data['response']['docs']

[{'doc': {'no': 1,
   'ranking': '1',
   'bookname': '우리가 빛의 속도로 갈 수 없다면 :김초엽 소설 ',
   'authors': '지은이: 김초엽',
   'publisher': '허블',
   'publication_year': '2019',
   'isbn13': '9791190090018',
   'addition_symbol': '03810',
   'vol': '',
   'class_no': '813.7',
   'class_nm': '문학 > 한국문학 > 소설',
   'loan_count': '487',
   'bookImageURL': 'https://image.aladin.co.kr/product/19359/16/cover/s972635417_1.jpg',
   'bookDtlUrl': 'https://data4library.kr/bookV?seq=5430429'}},
 {'doc': {'no': 2,
   'ranking': '2',
   'bookname': '달러구트 꿈 백화점 :이미예 장편소설',
   'authors': '지은이: 이미예',
   'publisher': '팩토리나인',
   'publication_year': '2020',
   'isbn13': '9791165341909',
   'addition_symbol': '03810',
   'vol': '',
   'class_no': '813.7',
   'class_nm': '문학 > 한국문학 > 소설',
   'loan_count': '407',
   'bookImageURL': 'https://image.aladin.co.kr/product/24512/70/cover/k392630952_1.jpg',
   'bookDtlUrl': 'https://data4library.kr/bookV?seq=5707051'}},
 {'doc': {'no': 3,
   'ranking': '3',
   'bookname': '지구에서 한

In [29]:
books = []
for d in data['response']['docs']:
    books.append(d['doc'])

In [30]:
books = [d['doc'] for d in data['response']['docs']]

In [31]:
books

[{'no': 1,
  'ranking': '1',
  'bookname': '우리가 빛의 속도로 갈 수 없다면 :김초엽 소설 ',
  'authors': '지은이: 김초엽',
  'publisher': '허블',
  'publication_year': '2019',
  'isbn13': '9791190090018',
  'addition_symbol': '03810',
  'vol': '',
  'class_no': '813.7',
  'class_nm': '문학 > 한국문학 > 소설',
  'loan_count': '487',
  'bookImageURL': 'https://image.aladin.co.kr/product/19359/16/cover/s972635417_1.jpg',
  'bookDtlUrl': 'https://data4library.kr/bookV?seq=5430429'},
 {'no': 2,
  'ranking': '2',
  'bookname': '달러구트 꿈 백화점 :이미예 장편소설',
  'authors': '지은이: 이미예',
  'publisher': '팩토리나인',
  'publication_year': '2020',
  'isbn13': '9791165341909',
  'addition_symbol': '03810',
  'vol': '',
  'class_no': '813.7',
  'class_nm': '문학 > 한국문학 > 소설',
  'loan_count': '407',
  'bookImageURL': 'https://image.aladin.co.kr/product/24512/70/cover/k392630952_1.jpg',
  'bookDtlUrl': 'https://data4library.kr/bookV?seq=5707051'},
 {'no': 3,
  'ranking': '3',
  'bookname': '지구에서 한아뿐 :정세랑 장편소설 ',
  'authors': '지은이: 정세랑',
  'publisher'

In [32]:
books_df = pd.DataFrame(books)

books_df

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,487,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429
1,2,2,달러구트 꿈 백화점 :이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,407,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,396,https://image.aladin.co.kr/product/19804/82/co...,https://data4library.kr/bookV?seq=5446546
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,393,https://image.aladin.co.kr/product/24131/37/co...,https://data4library.kr/bookV?seq=5674475
4,5,5,아몬드손원평 장편소설,손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,390,http://image.aladin.co.kr/product/16839/4/cove...,https://data4library.kr/bookV?seq=3423096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,194,열두 발자국,지은이: 정재승,어크로스,2018,9791160560510,03400,,404,"자연과학 > 자연과학 > 강연집, 수필집, 연설문집",80,https://image.aladin.co.kr/product/15072/28/co...,https://data4library.kr/bookV?seq=3767852
196,197,197,죽음 :베르나르 베르베르 장편소설,지은이: 베르나르 베르베르 ;옮긴이: 전미연,열린책들,2019,9788932919676,04860,1,863,문학 > 프랑스문학 > 소설,79,https://image.aladin.co.kr/product/19183/62/co...,https://data4library.kr/bookV?seq=5408151
197,198,197,심판,베르나르 베르베르 지음 ;전미연 옮김,열린책들,2020,9788932920405,03860,,862,문학 > 프랑스문학 > 희곡,79,https://image.aladin.co.kr/product/24919/70/co...,https://data4library.kr/bookV?seq=5738734
198,199,197,어떤 죽음이 삶에게 말했다 :생의 남은 시간이 우리에게 들려주는 것,김범석 지음,흐름출판,2021,9788965964230,03810,,814.7,문학 > 한국문학 > 수필,79,https://image.aladin.co.kr/product/26120/62/co...,https://data4library.kr/bookV?seq=5850509


In [33]:
books_df.to_json('20s_best_book.json')